In [3]:
import pandas as pd
import json

def process_trades(json_file, ticker, exchange, algo=None, interval=None, mode='all'):
    # Load the data from the JSON file
    target_user_id = "6673d10b68af094502c93931"
    with open(json_file) as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    df = pd.json_normalize(data)

    # Define the start and end times for filtering
    start_time = pd.to_datetime('13:30:00').time()
    end_time = pd.to_datetime('20:00:00').time()

    # Filter alerts for the given ticker and interval
    alerts = df[
        (df['symbol'] == ticker) &
        (df['user_id.$oid'] == target_user_id)
    ]
    
    if interval:
        alerts = alerts[alerts['interval'] == interval]
    
    # If the mode is not 'algo_summary', apply the algo filter
    if algo and mode != 'algo_summary':
        alerts = alerts[alerts['algo'] == algo]
    
    # Extract the nested datetime value and convert it to datetime
    alerts['alerted_at'] = pd.to_datetime(alerts['alerted_at.$date'])

    # Ensure that the conversion was successful
    if alerts['alerted_at'].isna().any():
        print("Warning: Some 'alerted_at' values could not be converted to datetime.")
    
    # Filter based on alerted_at between start and end times
    if exchange == 'SMART':
        filtered_alerts = alerts[(alerts['alerted_at'].dt.time >= start_time) & (alerts['alerted_at'].dt.time <= end_time)]
    else:
        filtered_alerts = alerts
    # Sort the alerts by date in descending order
    filtered_alerts = filtered_alerts.sort_values(by='alerted_at', ascending=False)
    
    # Remove the last trade of each day
    filtered_alerts['date'] = filtered_alerts['alerted_at'].dt.date
    #display(filtered_alerts)
    #filtered_alerts = filtered_alerts.groupby(['algo', 'date']).apply(lambda x: x.iloc[1:]).reset_index(drop=True)
    
    # Define the function to calculate profit/loss
    def calculate_profit_loss(row):
        if row['order_status'] in ["order_failed", "order_running"]:
            return row

        buying_total = 0
        selling_total = 0

        for event in row['events']:
            side = event['side']
            contracts = event['contracts']
            price = float(event['price'])

            if side == 'SELL':
                selling_total += contracts * price
            elif side == 'BUY':
                buying_total += contracts * price

        profit_or_loss = selling_total - buying_total
        row['profitOrLoss'] = profit_or_loss
        row['profitOrLossPercentage'] = (profit_or_loss * 100) / 500
        return row

    # Apply the function to calculate profit/loss for each trade
    filtered_alerts = filtered_alerts.apply(calculate_profit_loss, axis=1)

    if mode == 'daily':
        # Group by date and calculate daily profit
        daily_profit = filtered_alerts.groupby('date').agg(
            profitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        daily_profit['profitOrLossPercentage'] = daily_profit['profitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")

        # Display the daily profits
        pd.set_option('display.max_rows', None)
        display(daily_profit)
        
        # Print total profit for the period
        total_daily_profit = daily_profit['profitOrLoss'].sum()
        total_trades = daily_profit['numberOfTrades'].sum()
        print(f"Total Daily Profit: {total_daily_profit}")
        print(f"Total Number of Trades: {total_trades}")
        
    elif mode == 'overall':
        # Calculate total profit and percentage
        total_profit = filtered_alerts['profitOrLoss'].sum()
        percentage_total_profit = (total_profit * 100) / 500  # Assuming 500 as total investment
        total_trades = filtered_alerts.shape[0]
        
        # Print calculated values
        print(f"Total Profit: {total_profit}")
        print(f"Percentage Total Profit: {percentage_total_profit}%")
        print(f"Total Number of Trades: {total_trades}")
    
    elif mode == 'all':
        # Print calculated values
        columns_to_select = ['symbol', 'alerted_at', 'alert_type', 'profitOrLoss', 'profitOrLossPercentage', 'entry.price', 'exit.price', 'order_status', 'algo', 'interval']
        
        filtered_alerts = filtered_alerts[columns_to_select]
        display(filtered_alerts)
    
    elif mode == 'algo_summary':
        # Group by algorithm and calculate total profit/loss and profit/loss percentage
        algo_summary = filtered_alerts.groupby('algo').agg(
            totalProfitOrLoss=('profitOrLoss', 'sum'),
            numberOfTrades=('symbol', 'size')
        ).reset_index()
        
        # Calculate profit/loss percentage based on total investment (assuming 500 as total investment per algo)
        algo_summary['profitOrLossPercentage'] = algo_summary['totalProfitOrLoss'].apply(lambda x: f"{(x * 100 / 500):.2f}%")
        
        # Display the summary table
        display(algo_summary)
        # Print total profit and total trades
        total_profit = algo_summary['totalProfitOrLoss'].sum()
        total_trades = algo_summary['numberOfTrades'].sum()
        print(f"Total Profit Across All Algorithms: {total_profit}")
        print(f"Total Number of Trades Across All Algorithms: {total_trades}")


In [11]:

#MARA, CLSK, COIN, RIOT, HUT, AMC, GME, AI, UPST, NKLA, KOSS, QS, DJT, CVNA, AFRM, PLTR, SOFI, PLUG, BLINK, NIO, TLRY, LCID, BYND, JWN, JMIA, TSLA
#TSLA - 5, EURUSD - 10, XAUUSD - 10, QQQ - 15, SPX - 15, SPY - 3_, SPY - 3, SPY - 15
# AFRM-Algo-1-3M-LS
ticker = 'HUT'
algo = 'Zero-AI-1'
slippage = '5'
exchange = 'SMART'
print("======Processing algo_summary=====")
process_trades('project_zero.running_trades.json', ticker, exchange, algo, slippage, 'algo_summary')
#print("======Processing daily=====")
process_trades('project_zero.running_trades.json', ticker, exchange,  algo, slippage, 'daily')
print("======Processing all=====")
process_trades('project_zero.running_trades.json', ticker, exchange,  algo, slippage, "all")

======Processing algo_summary=====


,algo,totalProfitOrLoss,numberOfTrades,profitOrLossPercentage
0,Zero-AI-1,172.0776,27,34.42%


Total Profit Across All Algorithms: 172.0775999999995
Total Number of Trades Across All Algorithms: 27


,date,profitOrLoss,numberOfTrades,profitOrLossPercentage
0,2024-11-07,6.2000,1,1.24%
1,2024-11-08,55.7064,4,11.14%
2,2024-11-12,90.3950,1,18.08%
3,2024-11-13,41.9100,1,8.38%
4,2024-11-15,10.9400,1,2.19%
5,2024-11-18,-20.3600,3,-4.07%
6,2024-11-19,-6.1200,2,-1.22%
7,2024-11-20,-37.4400,1,-7.49%
8,2024-11-21,-22.9050,3,-4.58%
9,2024-11-22,24.0153,1,4.80%


Total Daily Profit: 172.0775999999995
Total Number of Trades: 27
======Processing all=====


,symbol,alerted_at,alert_type,profitOrLoss,profitOrLossPercentage,entry.price,exit.price,order_status,algo,interval
2225,HUT,2024-11-29 17:00:12.503000+00:00,Short,NaN,NaN,28.64,0,order_running,Zero-AI-1,5
2102,HUT,2024-11-27 17:40:03.707000+00:00,Long,31.8200,6.36400,27.28,28.64,order_closed,Zero-AI-1,5
2077,HUT,2024-11-27 16:10:01.945000+00:00,Short,-25.0654,-5.01308,25.9927,27.0825,order_closed,Zero-AI-1,5
2009,HUT,2024-11-27 14:35:05.787000+00:00,Long,26.3631,5.27262,24.825,25.9927,order_closed,Zero-AI-1,5
1954,HUT,2024-11-26 18:25:04.182000+00:00,Short,7.7400,1.54800,24.87,24.825,order_closed,Zero-AI-1,5
1916,HUT,2024-11-26 16:30:11.225000+00:00,Long,-11.1550,-2.23100,25.355,24.87,order_closed,Zero-AI-1,5
1833,HUT,2024-11-25 19:30:00.424000+00:00,Short,7.3200,1.46400,24.9,24.78,order_closed,Zero-AI-1,5
1771,HUT,2024-11-25 15:50:03.541000+00:00,Long,-13.4320,-2.68640,25.484,24.9,order_closed,Zero-AI-1,5
1717,HUT,2024-11-25 14:40:05.462000+00:00,Short,6.1452,1.22904,24.56,25.0279,order_closed,Zero-AI-1,5
1629,HUT,2024-11-22 15:45:11.062000+00:00,Long,24.0153,4.80306,23.75,24.56,order_closed,Zero-AI-1,5


In [3]:
import pandas as pd
import json

# Helper function to calculate profit/loss for each trade
def calculate_profit_loss(row):
    if row['order_status'] in ["order_failed", "order_running"]:
        return row

    buying_total = 0
    selling_total = 0

    for event in row['events']:
        side = event['side']
        contracts = event['contracts']
        price = float(event['price'])

        if side == 'SELL':
            selling_total += contracts * price
        elif side == 'BUY':
            buying_total += contracts * price

    profit_or_loss = selling_total - buying_total
    row['profitOrLoss'] = profit_or_loss
    row['profitOrLossPercentage'] = (profit_or_loss * 100) / 500  # Assuming 500 as total investment
    return row

# Main function to process trades for multiple tickers and create a summary
def process_trades_for_summary(json_file, tickers, exchange, algo=None, interval=None, mode='algo_summary'):
    # Load data from JSON file
    target_user_id = "6673d10b68af094502c93931"
    with open(json_file) as f:
        data = json.load(f)
    
    # Convert JSON data to DataFrame
    df = pd.json_normalize(data)
    
    # Initialize an empty list to collect each ticker's result
    results = []

    # Loop through each ticker and process trades
    for ticker in tickers:
        # Filter by ticker and target user
        alerts = df[(df['symbol'] == ticker) & (df['user_id.$oid'] == target_user_id)]
        
        if interval:
            alerts = alerts[alerts['interval'] == interval]
        if algo and mode != 'algo_summary':
            alerts = alerts[alerts['algo'] == algo]
        
        # Convert 'alerted_at' to datetime
        alerts.loc[:, 'alerted_at'] = pd.to_datetime(alerts['alerted_at.$date'])

        # Time filtering if exchange is SMART
        start_time = pd.to_datetime('13:30:00').time()
        end_time = pd.to_datetime('20:00:00').time()
        
        if exchange == 'SMART':
            alerts = alerts[(alerts['alerted_at'].dt.time >= start_time) & (alerts['alerted_at'].dt.time <= end_time)]

        # Sort alerts by date
        alerts = alerts.sort_values(by='alerted_at', ascending=False)

        # Apply the profit/loss calculation function
        alerts = alerts.apply(calculate_profit_loss, axis=1)
        
        # Summary for 'algo_summary' mode
        if mode == 'algo_summary':
            algo_summary = alerts.groupby('algo').agg(
                totalProfitOrLoss=('profitOrLoss', 'sum'),
                numberOfTrades=('symbol', 'size')
            ).reset_index()
            
            algo_summary['profitOrLossPercentage'] = algo_summary['totalProfitOrLoss'].apply(
                lambda x: f"{(x * 100 / 500):.2f}%"
            )
            # Add a column for ticker
            algo_summary['ticker'] = ticker
            
            # Append to results list
            results.append(algo_summary)

    # Concatenate all results into a single DataFrame
    summary_table = pd.concat(results, ignore_index=True)
    
    # Display the summary table
    display(summary_table)
    
    # Return the summary table if needed for further processing
    return summary_table


# List of tickers to summarize
tickers = ['MARA', 'CLSK', 'COIN', 'RIOT', 'HUT', 'AMC', 'GME', 'AI', 'UPST', 'NKLA', 'KOSS', 
           'QS', 'DJT', 'CVNA', 'AFRM', 'PLTR', 'SOFI', 'PLUG', 'BLINK', 'NIO', 'TLRY', 'LCID', 
           'BYND', 'JWN', 'JMIA', 'TSLA']

# Call the function to generate the summary table for all tickers
summary_table = process_trades_for_summary(
    'project_zero.running_trades.json', tickers, exchange='SMART', algo='Zero-AI-1', mode='algo_summary'
)


/var/folders/5w/dyscq3150dx65vqv6d_qgtsm0000gn/T/ipykernel_44843/2597195447.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alerts.loc[:, 'alerted_at'] = pd.to_datetime(alerts['alerted_at.$date'])
/var/folders/5w/dyscq3150dx65vqv6d_qgtsm0000gn/T/ipykernel_44843/2597195447.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alerts.loc[:, 'alerted_at'] = pd.to_datetime(alerts['alerted_at.$date'])
/var/folders/5w/dyscq3150dx65vqv6d_qgtsm0000gn/T/ipykernel_44843/2597195447.py:51: SettingWithCopyWarning: 


KeyError: "Column(s) ['profitOrLoss'] do not exist"

In [148]:
# New mode: 'ticker_summary'
process_trades_2('project_zero.running_trades.json', ticker='MARA', exchange='SMART', algo='Zero-AI-3', interval='1', mode='ticker_summary')


,symbol,totalProfitOrLoss,numberOfTrades,profitOrLossPercentage
0,MARA,149.538,709,29.91%


Total Profit Across All Tickers: 149.53799999999535
Total Number of Trades Across All Tickers: 709
